# Нейросеть различающая котов и собак
https://www.youtube.com/watch?v=WmNmILXPj2Y
1:30 минута

In [1]:
%pip install torch torchvision numpy matplotlib opencv-python tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision as tv


import numpy as np
import matplotlib.pyplot as plt
import cv2 
import os

from tqdm import tqdm

c:\Users\muzaf\anaconda3\envs\dog\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class Dataset2class(torch.utils.data.Dataset):
    def __init__(self, path_dir1:str, path_dir2:str):
        super().__init__()

        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2

        self.dir1_list = sorted(os.listdir(path_dir1))
        self.dir2_list = sorted(os.listdir(path_dir2))
    def __len__(self):
        return  len(self.dir1_list) + len(self.dir2_list)

    def __getitem__(self,idx):
        
        if idx <= len(self.dir1_list):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.dir1_list[idx])
        else:
            class_id = 1
            idx -= len(self.dir_list)
            img_path = os.path.join(self.path_dir2, self.dir2_list[idx])

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = img[:, :, ::-1]


        img = img.astype(np.float32)
        img = img/255

        img = cv2.resize(img, (64,64), interpolation = cv2.INTER_AREA)
        img.transpose((2, 0, 1))

        t_img = torch.from_numpy(img)
        t_class_id = torch.tensor(class_id)
    
        return {'img': t_img, 'label': t_class_id}
        

In [4]:
train_dogs_path = 'C:/Users/muzaf/Desktop/SUAI_AI/Cats and Dogs/training_set/dogs/'
train_cats_path = 'C:/Users/muzaf/Desktop/SUAI_AI/Cats and Dogs/training_set/cats/'

test_dogs_path = 'C:/Users/muzaf/Desktop/SUAI_AI/Cats and Dogs/test_set/dogs/'
test_cats_path = 'C:/Users/muzaf/Desktop/SUAI_AI/Cats and Dogs/test_set/cats/'

train_ds_catsdogs = Dataset2class(train_dogs_path, train_cats_path)
test_ds_catsdogs = Dataset2class(test_dogs_path, test_cats_path)

In [5]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    train_ds_catsdogs, shuffle=True, batch_size = batch_size, num_workers=1, drop_last=True
)
test_loader = torch.utils.data.DataLoader(
    test_ds_catsdogs, shuffle=True, batch_size = batch_size, num_workers=1, drop_last=False
)

In [7]:
class ConvNet (nn.Module):
    def __init__(self):
        super().__init__()
        
        self.act = nn.LeakyReLU(0.2)
        self.maxpool = nn.MaxPool2d(2,2)
        self.conv0 = nn.Conv2d(3,32,3, stride=1, padding=0)
        self.conv1 = nn.Conv2d(32,32,3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32,32,3, stride=1, padding=0)
        self.conv3 = nn.Conv2d(64,64,3, stride=1, padding=0)
        self.conv4 = nn.Conv2d(64,64,3, stride=1, padding=0)

        self.adaptivepool = nn.AdaptiveAvgPool2d((1,1))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(64,10)
        self.linear2 = nn.Linear(10,2)

    def forward(self,x):

        out = self.conv0(x)
        out = self.act(out)
        out = self.maxpool(out)

        out = self.conv1(x)
        out = self.act(out)
        out = self.maxpool(out)
        
        out = self.conv2(x)
        out = self.act(out)
        out = self.maxpool(out)
       
        out = self.conv3(x)
        out = self.act(out)

        out = self.adaptivepool(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = self.act(out)
        out = self.linear2(out)
      
        print(out.shape)
        return out

In [8]:
model = ConvNet()

In [9]:
model

ConvNet(
  (act): LeakyReLU(negative_slope=0.2)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (adaptivepool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=64, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=2, bias=True)
)

In [10]:
for sample in train_loader:
    img = sample['img']
    label = sample['label']
    model(img)
    break

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9,0.999))